# Red Hat OpenShift Data Science + Starburst Galaxy:  <br>  Access and Explore your Data

## Introduction
**Red Hat OpenShift Data Science**, a managed cloud platform for Data Scientists and Developers of intelligent applications, supports the full Machine Learning lifecycle by providing a robust, scalable platform and a flexible, interactive environment for teams to do their work. **Starburst Galaxy**, based on open source Trino (formerly PrestoSQL), is a managed service providing a single point of access to your data without having to move it to a central repository. **Starburst Galaxy** focuses on the first, and often most difficult problem teams face when starting a new project - **The Acquisition and Preparation of Data**. 

This demonstration illustrates how quickly a Data Scientist can access data and pull it into the Red Hat OpenShift Data Science Jupyter environment using Starburst Galaxy.


**Figure:** Machine Learning Lifecycle

![MLLC](files/ml-lifecycle-desktop.svg)



**Useful links**:
* [Red Hat OpenShift Data Science](https://www.redhat.com/en/technologies/cloud-computing/openshift/openshift-data-science)
* [Starburst Galaxy](https://www.starburst.io/platform/starburst-galaxy/)
* [AI/ML on OpenShift](https://cloud.redhat.com/learn/topics/ai-ml)

## Installing Required Packages
Red Hat OpenShift Data Science provides images loaded with popular Open Source Data Science packages. These are also the notebook images we use when building our own intelligent applications. 

While these images typically have everything we need, we can always layer in specific package requirements using `pip` and a `requirements.txt` file. 

In [ ]:
!pip install -U -r requirements.txt

## Environment Initialization

### Import packages and environment variables
Let's import the packages we installed in the previous step and a simple helper function included in `helper.py`. Let's also assign the environment variables we included while spawning our notebook. This way, we won't accidentally expose sensitive connection information!

Lastly, we use these variables and the `trino.dbapi.connect` function to create our Connection object. 

### A note on security
Please ensure your Starburst Galaxy cluster meets your specific security requirements. 
There are three general areas to consider: 
- Securing access to the cluster
- Securing internal traffic within the cluster
- Securing data source connections

By default, Starburst Galaxy client access is secured using HTTPS/TLS.
Authentication can be configured using your preffered authentication provider, i.e. LDAP. 
Lastly, enable authorization and access controls. 
**Source**: [Security Overview](https://docs.starburst.io/356-e/security/overview.html#security-client)

For the purposes of this demonstration, let's assume HTTPS/TLS and Basic Authentication is sufficient.

In [ ]:
import os
from helper import get_sql 
import trino

TRINO_USERNAME = os.environ.get('TRINO_USERNAME')
TRINO_PASSWORD = os.environ.get('TRINO_PASSWORD')
TRINO_HOSTNAME = os.environ.get('TRINO_HOSTNAME')
TRINO_PORT     = os.environ.get('TRINO_PORT')

In [ ]:
conn = trino.dbapi.connect(
    host=TRINO_HOSTNAME,
    port=TRINO_PORT,
    user=TRINO_USERNAME,
    http_scheme='https',
    auth=trino.auth.BasicAuthentication(TRINO_USERNAME, TRINO_PASSWORD),
)

## What is Starburst Galaxy and how does it work? 
Starburst Galaxy is an incredibly efficient layer sitting between the data consumer and our data sources. It brings our data together so we can query sources individually or join them together in ways that previously required extensive ETL processes.

Let's use our connection object and SQL statements to interact with Starburst Galaxy. 

The following SQL statements help us understand our data sources:
* `'SHOW CATALOGS'` shows the data sources available to us at this time,
* `'SHOW SCHEMAS'` indicates how data tables are organized, and 
* `'SHOW TABLES'` exposes datasets within a catalog and schema.

In [ ]:
sql = 'SHOW CATALOGS'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'SHOW SCHEMAS from tpch'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = 'SHOW TABLES FROM tpch.sf1'
df = get_sql(sql, conn)
df.head()

**Please note**:
Our example here only includes the sample datasets shipped with Starburst Galaxy. Your data sources will show up under `'SHOW CATALOGS'` after their respective connectors are configured in your catalog.

**Some useful terminology**:
* **data source** - your data stored in a database, bucket, or other. Starburst has connectors for most sources already. 
* **connector** - connectors configure your catalog. They give you access to your data sources. They are similar to drivers, in a way.
* **catalog** - defines schemas and properties of connections so Galaxy knows how to query your data.
* **schema**  - how your tables are organized.
* **table**   - similar to tables in a relational database. A set of rows and columns representing your data based on connector properties.
* **tpch**    - a synthetic dataset used to benchmark systems. We are using it here for a sample dataset.

**Useful links**:
[Starburst Documentation](https://docs.starburst.io/)

## Accessing Data
We've seen the data sources we have access to (CATALOGS), their associated SCHEMAS, and some data tables from tpch.  Let's run some actual queries to access the data!

In [ ]:
sql = 'SELECT * FROM tpch.sf1.customer limit 40'
df = get_sql(sql, conn)
df.head()

In [ ]:
sql = "SELECT * FROM tpch.sf1.orders limit 40"
df = get_sql(sql, conn)
df.head()

And like other relational databases, we can join tables together for more useful views.

In [ ]:
sql = """

SELECT 
    c.custkey, 
    c.name, 
    c.nationkey, 
    c.mktsegment, 
    o.totalprice, 
    o.orderdate 
FROM tpch.sf1.customer c 
JOIN tpch.sf1.orders o ON c.custkey = o.custkey 
ORDER BY o.orderdate DESC
limit 40

"""
df = get_sql(sql, conn)
df.head()

## What's Next
Starburst Galaxy's real power is its ability to access multiple data sources using a single query and push processing down to the analytical store housing your data. However, we must first connect a data source to your Starburst Galaxy cluster which is outside the scope of our tutorial today.  

Let me highlight this capability with a simple example. Let's assume you connect a data source containing marketing campaign data to your catalog. From there, you could run the following hypothetical query joining your `email` marketing compaign to your `customer` and `order` tables. 

In [ ]:
# this code is illustrative only
sql = """

SELECT 
    c.custkey, 
    c.name, 
    c.mktsegment, 
    SUM(o.totalprice) as sum_ordered,
    e.open_rate
FROM tpch.sf1.customer c 
JOIN tpch.sf1.orders o ON c.custkey = o.custkey 
JOIN marketing.campaigns.email e ON e.custkey = c.custkey
GROUP BY c.custkey
ORDER BY o.orderdate DESC
limit 40

"""

**That's two disparate data sources behaving as if they are in the same data warehouse!**

## Conclusion
There's no question that this demo barely scratches the surface of what **Red Hat OpenShift Data Science** and **Starburst Galaxy** have to offer. We hope it has shown you, at least at a highlevel, how these tools enable data acccess and exploration so Data Science teams can quickly advance to the next phase in the Machine Learning Lifecycle - **Model Building**.